In [2]:
import pandas as pd
from pathlib import Path
import datetime
from datetime import time

Needs:
x Create df for one user  
save night points to a list  
save day points to a list  
cluster these points  
home will be cluster with most day points  
work will be cluster with most night points  

convert from gmt to local time using gps coords  

In [3]:
user_id = '000'
data_path = Path('../data/raw')
user_path = data_path / user_id / 'Trajectory'

def read_plt_file(filepath):
    """Read a single PLT trajectory file"""
    df = pd.read_csv(
        filepath,
        skiprows=6,
        header=None,
        names=['lat', 'lon', 'zero', 'altitude', 'days', 'date', 'time']
    )
    
    # Combine date and time into datetime
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])

    df['hour'] = df['datetime'].dt.hour

    # Add a column to add name of trajectory file
    df['filename'] = filepath.name

    # Add user name
    df['user'] = user_id
    
    # Drop unnecessary columns
    df = df[['user', 'lat', 'lon', 'altitude', 'datetime', 'hour', 'filename']]
    
    return df

# Get their trajectory files
trajectory_files = list(user_path.glob('*.plt'))
all_traj_files = []

for traj_file in trajectory_files:
    traj = read_plt_file(traj_file)
    all_traj_files.append(traj)

df = pd.concat(all_traj_files, ignore_index=True)


Save night points and day points to df
all times are saved in gmt, beijing time is gmt + 8
want to implement a way later to convert time according to gps coords

night points in gmt: 23:00 to 7:00
day points in gmt: 9:00 to 17:00

subtracting 8 hrs for beijing time
night points: 15:00 to 23:00
day points: 1:00 to 9:00

In [4]:
print(type(df['datetime'][0]))

<class 'pandas.Timestamp'>


Want to add that day points are only for mon-fri. What I have below is for 
all 7 days

In [5]:

night_points = df[df['hour'].between(15, 23)]
day_points = df[df['hour'].between(1, 9)]

use hdbscan to cluster 

In [6]:
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=5)

coords = night_points[['lat', 'lon']].values
clusters = clusterer.fit_predict(coords)

In [7]:
night_points['cluster'] = clusters

# Count points per cluster (exclude -1 which is noise)
valid_clusters = night_points[night_points['cluster'] != -1]
home_cluster = valid_clusters['cluster'].value_counts().idxmax()

home_location = night_points[night_points['cluster'] == home_cluster][['lat', 'lon']].mean()

In [8]:
print(f"Home location: {home_location['lat']}, {home_location['lon']}")

Home location: 40.004885519163764, 116.30801704878051


Try to find work location with day points

In [9]:
coords = day_points[['lat', 'lon']].values
clusters = clusterer.fit_predict(coords)

day_points['cluster'] = clusters

# Count points per cluster (exclude -1 which is noise)
valid_clusters = day_points[day_points['cluster'] != -1]
work_cluster = valid_clusters['cluster'].value_counts().idxmax()

work_location = day_points[day_points['cluster'] == work_cluster][['lat', 'lon']].mean()

print(f"Work location: {work_location['lat']}, {work_location['lon']}")


Work location: 31.144915580952382, 121.2104172952381


Work location is showing in shanghai. This needs more work. This will plot the 
clusters in folium

In [10]:
import folium

m = folium.Map(location=[work_location['lat'], work_location['lon']], zoom_start=12, tiles='CartoDB positron')

# Plot clusters
for cluster_id in day_points['cluster'].unique():
    if cluster_id == -1:
        continue
    cluster_data = day_points[day_points['cluster'] == cluster_id]
    color = 'green' if cluster_id == work_cluster else 'blue'
    
    for _, row in cluster_data.iterrows():
        folium.CircleMarker(
            [row['lat'], row['lon']],
            radius=2,
            color=color,
            fill=True
        ).add_to(m)

# Mark detected work
folium.Marker(
    [work_location['lat'], work_location['lon']],
    popup='Detected Work',
    icon=folium.Icon(color='green', icon='briefcase')
).add_to(m)

m.save('../output/work_clusters.html')

All cluster points are in shanghai. Check the df

In [11]:
df

,user,lat,lon,altitude,datetime,hour,filename
0,000,39.984702,116.318417,492,2008-10-23 02:53:04,2,20081023025304.plt
1,000,39.984683,116.318450,492,2008-10-23 02:53:10,2,20081023025304.plt
2,000,39.984686,116.318417,492,2008-10-23 02:53:15,2,20081023025304.plt
3,000,39.984688,116.318385,492,2008-10-23 02:53:20,2,20081023025304.plt
4,000,39.984655,116.318263,492,2008-10-23 02:53:25,2,20081023025304.plt
...,...,...,...,...,...,...,...
173865,000,40.000403,116.327255,149,2009-07-05 07:44:55,7,20090705025307.plt
173866,000,40.000433,116.327209,150,2009-07-05 07:45:00,7,20090705025307.plt
173867,000,40.000443,116.327186,150,2009-07-05 07:45:05,7,20090705025307.plt
173868,000,40.000522,116.327132,149,2009-07-05 07:45:10,7,20090705025307.plt


In [12]:
day_points

,user,lat,lon,altitude,datetime,hour,filename,cluster
0,000,39.984702,116.318417,492,2008-10-23 02:53:04,2,20081023025304.plt,1988
1,000,39.984683,116.318450,492,2008-10-23 02:53:10,2,20081023025304.plt,1988
2,000,39.984686,116.318417,492,2008-10-23 02:53:15,2,20081023025304.plt,1988
3,000,39.984688,116.318385,492,2008-10-23 02:53:20,2,20081023025304.plt,1988
4,000,39.984655,116.318263,492,2008-10-23 02:53:25,2,20081023025304.plt,1988
...,...,...,...,...,...,...,...,...
173865,000,40.000403,116.327255,149,2009-07-05 07:44:55,7,20090705025307.plt,-1
173866,000,40.000433,116.327209,150,2009-07-05 07:45:00,7,20090705025307.plt,4039
173867,000,40.000443,116.327186,150,2009-07-05 07:45:05,7,20090705025307.plt,-1
173868,000,40.000522,116.327132,149,2009-07-05 07:45:10,7,20090705025307.plt,-1
